In [1]:
import numpy as np                     # Llibreria matematica
import matplotlib.pyplot as plt        # Per mostrar plots
import sklearn                         # Llibreia de DM
import sklearn.datasets as ds            # Per carregar mes facilment el dataset digits
import sklearn.model_selection as cv    # Pel Cross-validation
import sklearn.neighbors as nb           # Per fer servir el knn
from sklearn.model_selection import cross_val_score  
import sklearn.preprocessing as Pre
import warnings
warnings.filterwarnings('ignore')       #Per fer que els warnings no apareguin
%matplotlib inline         

In [2]:
import numpy as np    # Numeric and matrix computation
import pandas as pd   # Optional: good package for manipulating data 
import sklearn as sk  # Package with learning algorithms implemented

# Loading the dataset.
df = pd.read_csv('train.csv', sep=',', na_values="NA")

In [3]:
print(df['SalePrice'].unique())

X1 = df[df['SalePrice'] == 'Level1']
X2 = df[df['SalePrice'] == 'Level2']
X3 = df[df['SalePrice'] == 'Level3']
X4 = df[df['SalePrice'] == 'Level4']
X5 = df[df['SalePrice'] == 'Level5']

bdata = pd.DataFrame()

for i in range(3):
    bdata = bdata.append(X1, ignore_index = True)
bdata = bdata.append(X2.sample(frac=1/3), ignore_index = True)
bdata = bdata.append(X3, ignore_index = True)
for i in range(4):
    bdata = bdata.append(X4, ignore_index = True)
for i in range(10):
    bdata = bdata.append(X5, ignore_index = True)

bdata['SalePrice'].value_counts()

['Level2' 'Level4' 'Level1' 'Level3' 'Level5']


Level1    237
Level4    236
Level3    223
Level2    209
Level5    170
Name: SalePrice, dtype: int64

In [4]:
X = bdata.drop(columns='SalePrice')
X = X.drop(columns = 'Id')
Y = bdata.loc[:,'SalePrice']

le = Pre.LabelEncoder()
le.fit(Y)
Y = le.fit_transform(Y)
i = 0
for t in X.dtypes:
    if t == object or t == bool:
        le.fit(X.iloc[:,i])
        X.iloc[:,i] = le.fit_transform(X.iloc[:,i])
    i += 1

X_train, X_test, y_train, y_test = cv.train_test_split(X, Y, test_size=.3, random_state=1)

In [5]:
dt = pd.read_csv('test.csv', sep=',', na_values="NA")
X = dt.drop(columns='SalePrice')
X = X.drop(columns = 'Id')
Y = dt.loc[:,'SalePrice']

le = Pre.LabelEncoder()
le.fit(Y)
Y = le.fit_transform(Y)
i = 0
for t in X.dtypes:
    if t == object or t == bool:
        le.fit(X.iloc[:,i])
        X.iloc[:,i] = le.fit_transform(X.iloc[:,i])
    i += 1
X_test = X
y_test = Y

In [16]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
import sklearn.ensemble as Ens
import sklearn.model_selection as cv

In [17]:
# Build two classifiers

# Classifier 1 Linear SVM
knc = SVC(C=100,kernel='linear')
knc.fit(X_train, y_train)
pred=knc.predict(X_test)
res1=np.zeros(Y.shape)
res1[pred==Y]=1

# Classifier 2 Gradient Tree Boosting
lb5=[]
for nest in [1,2,5,10,20,50,100,200]:
    scores = cv.cross_val_score(Ens.GradientBoostingClassifier(n_estimators=nest, learning_rate=0.14),
                                X_train, y_train, cv=kfold, scoring='f1_macro')
    lb5.append(scores.mean())
    
bestgb = Ens.GradientBoostingClassifier(learning_rate=0.14, n_estimators=[1,2,5,10,20,50,100,200][np.argmax(lb5)])
pred = bestgb.fit(X_train, y_train).predict(X_test)
res2=np.zeros(Y.shape)
res2[pred==Y]=1

# Build contingency matrix
n00 = np.sum([res1[res2==1]==1])
n11 = np.sum([res1[res2==0]==0])
n10 = np.sum([res1[res2==1]==0])
n01 = np.sum([res1[res2==0]==1])

# Chi -square test
print("Have the classifiers significant different accuracy?:",(np.abs(n01-n10)-1)**2/(n01+n10)>3.84)

NameError: name 'kfold' is not defined